Ваш друг из соседнего отдела прислал вам файл, где он собирал информацию о сделках с компаниями в течение нескольких месяцев. К сожалению, друг был не особо аккуратен, и не знал, какой ужасный формат он выбрал... Ваша задача заключается в следующем: нужно собрать все данные в нормальный датафрэйм (и сказать другу, чтобы хранил данные аккуратнее).

Данные находятся в файле tables.xlsx,  представлены набором таблиц в одном эксельном листе.

In [2]:
import pandas as pd
import numpy as np

from io import BytesIO
import requests

In [153]:
df = pd.read_excel('d://Carpov_Courses/projects/dop_7/tables.xlsx')
df['mark'] = df['company41'].str.contains('company', regex=False)
df = df.rename(columns={'company41':'c41', 'company51':'c51','company50':'c50', 
                        'company76':'c76', 'company47':'c47', 'company27':'c27', 
                        'company48':'c48'})
df

,Unnamed: 0,c41,c51,c50,c76,c47,c27,c48,mark
0,NaN,580,198,968,220,788,554,479,NaN
1,NaN,786,271,69,566,NaN,193,636,NaN
2,NaN,578,NaN,526,74,984,581,951,NaN
3,NaN,614,NaN,589,575,85,122,454,NaN
4,NaN,44,849,992,155,527,33,NaN,NaN
5,NaN,859,966,918,312,NaN,404,407,NaN
6,NaN,225,486,20,367,339,481,252,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,company41,company51,company50,company76,company47,company27,company48,True


In [99]:
df[0:10]

,Unnamed: 0,company41,company51,company50,company76,company47,company27,company48,mark
0,NaN,580,198,968,220,788,554,479,NaN
1,NaN,786,271,69,566,NaN,193,636,NaN
2,NaN,578,NaN,526,74,984,581,951,NaN
3,NaN,614,NaN,589,575,85,122,454,NaN
4,NaN,44,849,992,155,527,33,NaN,NaN
5,NaN,859,966,918,312,NaN,404,407,NaN
6,NaN,225,486,20,367,339,481,252,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,company41,company51,company50,company76,company47,company27,company48,True


In [154]:
# находим индексы строк, где есть текст, т.е. mark = True
index = df.index
condition = df["mark"] == True
mark_indices = index[condition]
mark_indices

Int64Index([9, 18, 22, 28, 38], dtype='int64')

In [155]:
# добавляем индекс самой последней строки
mark_indices = list(mark_indices)
mark_indices.append(df.shape[0]-1)
mark_indices

[9, 18, 22, 28, 38, 43]

In [163]:
prev = 0 # начальный индекс для итерации по df
temp_temp_df = pd.DataFrame() # пустой df для первого объединения

for i in mark_indices:
    next_ = i # конечный индекс для итерации по df
    temp_df = df[prev:next_+1]
    prev = next_
    temp_df = temp_df.drop(temp_df[temp_df.mark == True].index)# выбросили строки с текстом
    
    if temp_df['Unnamed: 0'].sum() == 0:#проверяем есть ли в колонке Unnamed данные
        temp_df = temp_df.drop(['Unnamed: 0', 'mark'], axis=1).fillna(0)# убираем Unnamed и mark
        merged_df = pd.concat([temp_temp_df, temp_df])
        temp_temp_df = merged_df
    
    else:# в Unnamed есть данные
        #выбросим самую крайнюю колнку без данных
        #переименуем колонки таким образом, что названия сдвинутся влево
        
        temp_df = temp_df \
        .drop(['c48','mark'], axis=1) \
        .rename(columns={'Unnamed: 0':'c41', 'c41':'c51','c51':'c50','c50':'c76','c76':'c27','c27':'c48'})
        merged_df = pd.concat([temp_temp_df, temp_df])
        temp_temp_df = merged_df

In [164]:
merged_df.shape

(39, 7)

In [165]:
merged_df['c41'].sum()

14918